In [39]:
import pandas as pd
import random
import numpy as np

user_log = pd.read_csv('./cf_model.csv')

In [49]:
user_log

,user_idx,id
0,0,11602
1,0,6080
2,0,2385
3,0,6076
4,0,6077
...,...,...
241739,45586,4773
241740,45587,6600
241741,45587,5061
241742,45587,11955


In [41]:
grouped = user_log.groupby('user_idx').count()
grouped = grouped[~grouped['id'].isin([1, 2])].reset_index() # 게임기록이 1,2개뿐인 유저는 학습에 사용 불가 따라서 제거

In [42]:
grouped

,user_idx,id
0,0,5
1,2,9
2,3,7
3,4,10
4,5,9
...,...,...
39361,45582,6
39362,45584,5
39363,45585,4
39364,45586,4


In [43]:
# 딕셔너리 생성
user_dict = dict(zip(grouped['user_idx'], grouped['id']//3))

In [44]:
train_set = pd.DataFrame(columns=['user_idx', 'id'])
test_set = pd.DataFrame(columns=['user_idx', 'id'])

In [46]:
random.seed(42) # 시드 고정

for k, v in user_dict.items():
    tmp_df = user_log[user_log['user_idx'] == k]
    
    # 데이터프레임에서 user_idx 열을 리스트로 변환
    user_idx_list = tmp_df['id'].unique().tolist()

    # 랜덤으로 2개의 user_idx 선택
    random_user_idx = random.sample(user_idx_list, k=v)

    # 선택되지 않은 user_idx 추출
    left_user_idx = [user_idx for user_idx in user_idx_list if user_idx not in random_user_idx]

    # 해당 user_idx를 가지고 train-test set을 만들기
    tmp_train_df = tmp_df[tmp_df['id'].isin(left_user_idx)]
    tmp_test_df = tmp_df[tmp_df['id'].isin(random_user_idx)]
    
    train_set = pd.concat([train_set,tmp_train_df])
    test_set = pd.concat([test_set,tmp_test_df])

In [47]:
train_set

,user_idx,id
1,0,6080
2,0,2385
3,0,6076
4,0,6077
8,2,11935
...,...,...
241737,45586,511
241739,45586,4773
241740,45587,6600
241741,45587,5061


In [48]:
test_set

,user_idx,id
0,0,11602
6,2,4917
7,2,9200
10,2,9596
16,3,7300
...,...,...
241724,45582,9663
241727,45584,11100
241735,45585,11100
241738,45586,7667


In [50]:
train_set.to_csv('./train_set.csv', index=False)
test_set.to_csv('./test_set.csv', index=False)

In [ ]:
dfdfdf여기까지ㅇㄹㅇㄹㅇㄹ

In [ ]:
total_user_idx = user_log['user_idx'].unique()

In [ ]:
user_log['id'].value_counts()

id
10986    3454
2203     2146
3331     1427
10820    1323
8508     1264
         ... 
9829        1
6393        1
63          1
1236        1
12490       1
Name: count, Length: 9644, dtype: int64

In [ ]:
grouped = user_log.groupby('user_idx').count()

In [ ]:
grouped = grouped.sort_values(by='id', ascending=False)
grouped = grouped[~grouped['id'].isin([1, 2])].reset_index() # 게임기록이 1,2개뿐인 유저는 학습에 사용 불가 따라서 제거

In [ ]:
grouped

,user_idx,id
0,788,19
1,2729,19
2,5269,16
3,429,14
4,12894,14
...,...,...
39361,35187,3
39362,33664,3
39363,16815,3
39364,16850,3


In [ ]:
total_user_idx = grouped['user_idx'].unique()

In [ ]:
grouped['id'].value_counts()

id
3     9517
4     6213
10    5716
5     4511
9     3867
6     3492
8     3056
7     2964
11      22
12       3
19       2
14       2
16       1
Name: count, dtype: int64

전체 유저 39366명 / 이중 20%는 약 7873명

1. 상위 20%로 만들기

In [ ]:
sorted_grouped = grouped.sort_values(by='id', ascending=False)

In [ ]:
tmp = sorted_grouped.head(8800)

In [ ]:
tmp.head(10)

,id
user_idx,
788,19
2729,19
5269,16
429,14
12894,14
43258,12
14655,12
28889,12
490,11


In [ ]:
tmp.tail()

,id
user_idx,
7055,9
530,9
18162,9
27452,9
25735,9


In [ ]:
test_set_idx = tmp.index.get_level_values('user_idx').unique()

In [ ]:
test_set_idx

Index([  788,  2729,  5269,   429, 12894, 43258, 14655, 28889,   490, 12672,
       ...
       32004,   550,   553,   554,  7036,  7055,   530, 18162, 27452, 25735],
      dtype='int64', name='user_idx', length=8800)

2. 비율이 많이 분포하는 지점에서 20% 랜덤 샘플링

In [ ]:
tmp2 = user_log[user_log['id'] == '8']

In [ ]:
tmp2 = grouped[grouped['id'].isin([8, 9, 10])]

In [ ]:
tmp2

,id
user_idx,
16637,10
4888,10
4897,10
22807,10
26307,10
...,...
10143,8
42983,8
21892,8


In [ ]:
user_log[user_log['user_idx'] == 16637]

,user_idx,id
96597,16637,10820
96598,16637,10810
96599,16637,3855
96600,16637,3146
96601,16637,10016
96602,16637,10996
96603,16637,2841
96604,16637,4004
96605,16637,5631
96606,16637,11237


In [ ]:
tmp2

,user_idx,id


In [ ]:
tmp2 = tmp2.reset_index()
tmp2

,user_idx,id
0,2,9
1,4,10
2,5,9
3,6,10
4,8,10
...,...,...
12634,45559,10
12635,45562,10
12636,45564,10
12637,45568,8


In [ ]:
tmp2['id'].unique()

In [ ]:
tmp2['id'].unique()

array([ 9, 10,  8])

In [ ]:
# 랜덤 시드 고정
np.random.seed(42)

test_set_idx_2 = np.random.choice(tmp2['user_idx'].unique(), size=8800, replace=False)

In [ ]:
test_set_idx_2

array([39080,  2814, 28825, ...,   331,  4107, 27767])

In [ ]:
len(test_set_idx_2)

8800

3. 각 집단별로 20%씩 뽑기

In [ ]:
# 랜덤 시드 고정
np.random.seed(42)
test_set_idx_3 = []

for i in range(1,20):
    tmp3 = grouped[grouped['id'].isin([i])]
    tmp3 = tmp3.reset_index()
    #print(tmp3)
    sampled_df = tmp3.sample(frac=0.2, random_state=42)
    user_idx_list = sampled_df['user_idx'].tolist()
    if user_idx_list :
        test_set_idx_3.append(user_idx_list)
    #print(user_idx_list)
    #print(len(user_idx_list))
    #print('*************************')
    #for j in np.random.choice(tmp3['user_idx'].unique(), size=int(len(tmp3)*0.2), replace=False):
    #    test_set_idx_3.append(j)

In [ ]:
test_set_idx_3

[[21778,
  44839,
  25999,
  17974,
  4578,
  35836,
  28812,
  12637,
  7648,
  14149,
  38997,
  22341,
  16126,
  45122,
  30318,
  17254,
  36753,
  36922,
  33492,
  36236,
  31478,
  44042,
  29260,
  21040,
  33985,
  24465,
  37244,
  13744,
  34209,
  35371,
  43494,
  9655,
  33180,
  4173,
  2573,
  39506,
  18137,
  16518,
  37046,
  45067,
  22674,
  23808,
  19090,
  26604,
  45536,
  19290,
  3763,
  36730,
  13663,
  44350,
  45341,
  23496,
  16914,
  11687,
  23726,
  21506,
  8959,
  12222,
  32537,
  15135,
  15259,
  41700,
  8421,
  5580,
  35427,
  25480,
  3667,
  44906,
  17816,
  44859,
  38735,
  40428,
  38919,
  4518,
  42607,
  38262,
  28017,
  25227,
  11405,
  39563,
  1634,
  42112,
  20887,
  11026,
  21799,
  25509,
  7530,
  45506,
  4161,
  17619,
  19787,
  31189,
  9642,
  45262,
  22650,
  31012,
  40035,
  44892,
  39407,
  21483,
  17105,
  32338,
  20377,
  40836,
  8377,
  16486,
  39083,
  40964,
  25656,
  30869,
  21306,
  20851,
  27235,

In [ ]:
test_set_idx_3 = np.concatenate(test_set_idx_3).tolist()

In [ ]:
test_set_idx_3

[21778,
 44839,
 25999,
 17974,
 4578,
 35836,
 28812,
 12637,
 7648,
 14149,
 38997,
 22341,
 16126,
 45122,
 30318,
 17254,
 36753,
 36922,
 33492,
 36236,
 31478,
 44042,
 29260,
 21040,
 33985,
 24465,
 37244,
 13744,
 34209,
 35371,
 43494,
 9655,
 33180,
 4173,
 2573,
 39506,
 18137,
 16518,
 37046,
 45067,
 22674,
 23808,
 19090,
 26604,
 45536,
 19290,
 3763,
 36730,
 13663,
 44350,
 45341,
 23496,
 16914,
 11687,
 23726,
 21506,
 8959,
 12222,
 32537,
 15135,
 15259,
 41700,
 8421,
 5580,
 35427,
 25480,
 3667,
 44906,
 17816,
 44859,
 38735,
 40428,
 38919,
 4518,
 42607,
 38262,
 28017,
 25227,
 11405,
 39563,
 1634,
 42112,
 20887,
 11026,
 21799,
 25509,
 7530,
 45506,
 4161,
 17619,
 19787,
 31189,
 9642,
 45262,
 22650,
 31012,
 40035,
 44892,
 39407,
 21483,
 17105,
 32338,
 20377,
 40836,
 8377,
 16486,
 39083,
 40964,
 25656,
 30869,
 21306,
 20851,
 27235,
 11975,
 17193,
 7105,
 34505,
 21150,
 34208,
 42184,
 38421,
 29656,
 12904,
 19267,
 21722,
 4182,
 44308,
 2

In [ ]:
len(test_set_idx_3)

7871

In [ ]:
test_set = user_log[user_log['user_idx'].isin(test_set_idx_3)]
test_set = test_set.sort_values(by='user_idx', ascending=True)

In [ ]:
test_set

,user_idx,id
6,2,4917
7,2,9200
8,2,11935
9,2,2505
10,2,9596
...,...,...
241730,45584,4385
241733,45585,9681
241734,45585,9663
241732,45585,9649


In [ ]:
train_set_idx = [x for x in total_user_idx if x not in test_set_idx_3]

In [ ]:
train_set = user_log[user_log['user_idx'].isin(train_set_idx)]
train_set = train_set.sort_values(by='user_idx', ascending=True)

In [ ]:
train_set

,user_idx,id
0,0,11602
1,0,6080
2,0,2385
3,0,6076
4,0,6077
...,...,...
241737,45586,511
241741,45587,5061
241742,45587,11955
241740,45587,6600
